# Langchain LlamaIndex

## Lib import

In [ ]:
!pip install llama-index llama-index-core llama-index-embeddings-huggingface sentence-transformers torch
!pip install llama-index-vector-stores-chroma
!pip install llama-index-llms-langchain

In [ ]:
import os
import logging
import operator
from typing import List, Literal, Annotated, Optional, Union, Any
from typing_extensions import TypedDict
import chromadb
from chromadb.config import Settings
from pydantic import BaseModel, Field, validator

from langchain import PromptTemplate, LLMChain
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnablePassthrough,
    RunnableSerializable,
)
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_chroma import Chroma
from langchain_text_splitters import TokenTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.schema import Document
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_huggingface import HuggingFaceEmbeddings

from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver

## Init config and API key

In [ ]:
#
# key
#
from dotenv import load_dotenv

load_dotenv()

deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
silicon_api_key = os.getenv("SILICON_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
linkup_api_key = os.getenv("LINKUP_API_KEY")

In [ ]:
#
# config
#
import tomllib


def load_config(config_file):
    try:
        with open(config_file, "rb") as f:
            config = tomllib.load(f)
            return config
    except Exception as e:
        print(f"Load config file error: {e}")
        return None


# load config file
deepseek_llm_model = None
silicon_base_url = None
silicon_llm_model = None
huggingface_embed_model = None

config_data = load_config("../config/config.toml")
if config_data:
    log_level = config_data.get("log_level")
    if log_level:
        logging.basicConfig(level=log_level)

    # deepseek
    deepseek_llm_model = config_data.get("deepseek", {}).get("model")
    deepseek_llm_temperature = config_data.get("deepseek", {}).get("temperature")
    deepseek_llm_max_tokens = config_data.get("deepseek", {}).get("max_tokens")

    # silicon
    silicon_base_url = config_data.get("silicon", {}).get("base_url")
    silicon_llm_model = config_data.get("silicon", {}).get("model")

    # huggingface
    huggingface_embed_model = config_data.get("huggingface", {}).get("embed_model")


# deepseek
deepseek_llm_model = deepseek_llm_model or "deepseek-chat"

# silicon
silicon_base_url = silicon_base_url or "https://api.siliconflow.cn/v1"
silicon_llm_model = silicon_llm_model or "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# huggingface
huggingface_embed_model = (
    huggingface_embed_model or "sentence-transformers/all-MiniLM-L6-v2"
)

## LLM Init

In [ ]:
# init LLM mod
llm_deepseek = ChatDeepSeek(
    model=deepseek_llm_model,
    temperature=deepseek_llm_temperature or 0.3,
    max_tokens=deepseek_llm_max_tokens,
    timeout=None,
    top_p=0.9,
    frequency_penalty=0.7,
    presence_penalty=0.5,
    max_retries=3,
)

## LlamaIndex

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings, StorageContext, ServiceContext
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from langchain_community.retrievers.llama_index import LlamaIndexRetriever

In [ ]:
root_path = '../'
html_directory = os.path.join(root_path, "downloads/website/calerie/")
persist_directory = os.path.join(root_path, "var/llama_index/vector_db")
collection_id = "calerie-health"

In [ ]:
embeddings_model = HuggingFaceEmbedding(
    model_name=huggingface_embed_model,
    device="cpu",  # GPU is "cuda"
    max_length=256, 
)

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from transformers import AutoTokenizer

Settings.llm = llm_deepseek
Settings.embed_model = embeddings_model
Settings.chunk_size = 256
Settings.text_splitter = SentenceSplitter(
    chunk_size=200,
    chunk_overlap=20,
    tokenizer=AutoTokenizer.from_pretrained("bert-base-chinese")
)

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore

In [ ]:
chroma_client = chromadb.PersistentClient(path=persist_directory)
chroma_collection = chroma_client.get_or_create_collection(collection_id)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
documents = SimpleDirectoryReader(html_directory).load_data()
index = VectorStoreIndex.from_documents(
    documents,
    vector_store=vector_store,
    show_progress=True
)

In [ ]:
# Using LlamaIndex “no_text" mode may be similar to using langchain retriever
query_engine = index.as_query_engine(similarity_top_k=3, response_mode='no_text')
response = query_engine.query("What are the Alpha Hope?")
if hasattr(response, "response") and response.response:
    print("=== respons ===")
    print(response.response)
elif hasattr(response, "source_nodes") and response.source_nodes:
    print("=== source_nodes ===")
    for node in response.source_nodes:
        if node.score < 0.3: continue
        #print(node.get_content())
        print(f"""
            \rScore {node.score:.4f}
            \rContent: {node.text[:500]}
            """)
else:
    print("None respons")

In [ ]:
# LlamaIndex uses some LLM requests and can be time consuming.
# Maybe this model is suitable for using LlamaIndex in Langgraph agent apps.
query_engine = index.as_query_engine(similarity_top_k=1, response_mode='compact')
response = query_engine.query("What are the Alpha Hope?")
print(response)